In [0]:
# Sebastian Kuźmiński - sk149029
# załadowanie PySpark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count
import random
# spark = SparkSession.builder.appName("Markov").getOrCreate() - wykorzystywane w Google Colab

# funkcja tworząca model przejść Markowa, tworzy pary z aktualnego słowa i następnego, następnie zamienia je na DataFrame (dwie kolumny: current, next)
def markov_model(words): 
    pairs = [(words[i], words[i+1]) 
             for i in range(len(words)-1)]
    df = spark.createDataFrame(pairs, ["current", "next"])
    # obliczenie ile razy powtarzają się pary słów (current + next)
    probability_df = (
        df.groupBy("current", "next")
        .agg(count("*").alias("count"))
    )
    # obliczenie ile razy powtarza się słowo w kolumnie current
    total_df = (
        probability_df.groupBy("current")
        .agg(count("*").alias("total"))
    )
    # wyliczenie prawdopodobieństwa (count / total)    
    markov_df = (
        probability_df.join(total_df, "current")
        .withColumn("probability", col("count") / col("total"))
    )
    return markov_df

# funkcja losuje kolejne słowo na podstawie prawdopodobieństw (zamiana DataFrame utworzonego przez Spark na zwykłą pythonową listę wierszy w celu użycia przez random)
def next_state(current):
    rows = markov_df.filter(col("current") == current).collect()
    if not rows:
        return None
    nexts = [r["next"] for r in rows]
    probs = [r["probability"] for r in rows]
    return random.choices(nexts, weights=probs)[0]

# funkcja generuje kolejne słowa, do momentu aż osiągnie ustaloną długość lub kolejne słowo nie będzie istnieć
def generate_text(length, start):
    text = start
    current = start
    for _ in range(length - 1):
        nxt = next_state(current)
        if not nxt:
            break
        text += " " + nxt
        current = nxt
    return text

In [0]:
# WARIANT 1: krótki jednowierszowy tekst
text = "kiedy wymawiam słowo przyszłość, pierwsza sylaba odchodzi już do przeszłości. kiedy wymawiam słowo cisza, niszczę ją"
slowo_start = "kiedy"
print(f"Słowo start: {slowo_start}\n")

words = text.lower().split()
markov_df = markov_model(words)

for _ in range(5):
    print(generate_text(20, slowo_start))

Słowo start: kiedy

kiedy wymawiam słowo cisza, niszczę ją
kiedy wymawiam słowo przyszłość, pierwsza sylaba odchodzi już do przeszłości. kiedy wymawiam słowo cisza, niszczę ją
kiedy wymawiam słowo cisza, niszczę ją
kiedy wymawiam słowo przyszłość, pierwsza sylaba odchodzi już do przeszłości. kiedy wymawiam słowo cisza, niszczę ją
kiedy wymawiam słowo cisza, niszczę ją


In [0]:
# WARIANT 2: wielowierszowy tekst Wisławy Szymborskiej
wiersz = """
nic dwa razy się nie zdarza i nie zdarzy
z tej przyczyny zrodziliśmy się bez wprawy
i pomrzemy bez rutyny
choćbyśmy uczniami byli najtępszymi w szkole świata
"""
text = wiersz
slowo_start = "nic"
print(f"Słowo start: {slowo_start}\n")

words = text.lower().split()
markov_df = markov_model(words)

for _ in range(5):
    print(generate_text(20, slowo_start))


Słowo start: nic

nic dwa razy się nie zdarzy z tej przyczyny zrodziliśmy się nie zdarza i nie zdarza i pomrzemy bez wprawy
nic dwa razy się bez wprawy i nie zdarzy z tej przyczyny zrodziliśmy się nie zdarza i nie zdarzy z
nic dwa razy się bez rutyny choćbyśmy uczniami byli najtępszymi w szkole świata
nic dwa razy się nie zdarza i pomrzemy bez wprawy i pomrzemy bez wprawy i pomrzemy bez wprawy i nie
nic dwa razy się bez rutyny choćbyśmy uczniami byli najtępszymi w szkole świata


In [0]:
# WARIANT 3: plik spam.csv (pobrany z https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset)
# dodatkowe biblioteki w celu pobrania pliku z linku
import requests
import io
import pandas as pd

# pobranie pliku
url = "https://raw.githubusercontent.com/skuzminskisk/Markov/main/spam.csv"
response = requests.get(url)

# wczytanie pliku do bufora pamięci, następnie z bufora do DataFrame Spark przez Pandas
data_buffer = io.StringIO(response.text)
pdf = pd.read_csv(data_buffer, encoding="utf-8")
df = spark.createDataFrame(pdf)

# w pliku znajdują się dwie kolumny (v1, v2). w v1 znajdują się wiersze ze słowem "ham" oraz "spam". filtruje tylko "spam"
spam_df = df.filter(col("v1") == "spam")
# następnie pobieram przefiltrowane wiadomości z kolumny "v2"
messages = [row["v2"] for row in spam_df.collect() if row["v2"]]
# połączenie wiadomości w jeden tekst
text = " ".join(messages).lower()

words = text.lower().split()
markov_df = markov_model(words)

# nie wiem jakiego konkretnie słowa użyć jako start, losuje z dostępnych
# slowo_start = "tekst"
slowo_start = random.choice(words)
print(f"Słowo startowe: {slowo_start}\n")

for _ in range(5):
    print(generate_text(20, slowo_start))

Słowo startowe: nokia

nokia & take a date now all will be revealed. po box 114/14 tcr/w1 filthy films &xxx pics straight to
nokia tone of music to claim this is currently 500 pounds - double mins and u have important messages &
nokia 3510i colour flag & doublemins & nokia logo&pic message is brought to activate, just call germany for 1st week!
nokia to 86688 only five pounds of a top ringtone order, ref number 0844 861 85 85. no ads 150p.
nokia to receive search postcode this is currently 500 pounds - tells u have won a callback orno to text
